In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd 
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [ ]:
train = pd.read_csv("train.csv")
train

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,551,15806307,Trevisano,720,Spain,Male,38,5,114051.97,2,0,1,107577.29,0
1,6897,15709621,Martin,682,France,Female,54,4,62397.41,1,1,0,113088.60,1
2,4588,15619340,Palmer,672,France,Female,31,5,119903.67,1,1,1,132925.17,0
3,291,15620746,Napolitani,592,Spain,Female,40,4,104257.86,1,1,0,110857.33,0
4,1673,15646372,Yao,753,Spain,Male,42,5,120387.73,1,0,1,126378.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5345,15584532,Yu,568,France,Female,35,6,121079.60,2,1,1,124890.50,1
7996,5837,15606641,Liao,602,Germany,Female,45,7,145846.07,1,1,0,99276.02,0
7997,7335,15739692,Ferri,679,Spain,Female,43,5,132810.01,1,1,0,130780.85,1
7998,9552,15791373,Worsnop,715,France,Male,38,4,118729.45,1,0,0,95484.52,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

df = train.drop(columns = ["RowNumber", "CustomerId", "Surname"])#deleting irrelevant feature
le = LabelEncoder()
df['Geography'] = le.fit_transform(df['Geography'])#transform categorical feature
df = df.replace("Male", 0)
df = df.replace("Female", 1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,720,2,0,38,5,114051.97,2,0,1,107577.29,0
1,682,0,1,54,4,62397.41,1,1,0,113088.60,1
2,672,0,1,31,5,119903.67,1,1,1,132925.17,0
3,592,2,1,40,4,104257.86,1,1,0,110857.33,0
4,753,2,0,42,5,120387.73,1,0,1,126378.57,0
...,...,...,...,...,...,...,...,...,...,...,...
7995,568,0,1,35,6,121079.60,2,1,1,124890.50,1
7996,602,1,1,45,7,145846.07,1,1,0,99276.02,0
7997,679,2,1,43,5,132810.01,1,1,0,130780.85,1
7998,715,0,0,38,4,118729.45,1,0,0,95484.52,0


In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X = df.drop('Exited', axis=1)
y = df['Exited']
# making data balanced
#X_res, y_res = sm.fit_resample(X, y)
#X_res.shape, y_res.shape

((12736, 10), (12736,))

In [ ]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit

param_grid = {"max_depth": [int(i) for i in range(3, 9)], "learning_rate": [float(j/100) for j in range(30)]}#define parameter range
cv = ShuffleSplit(n_splits = 5, test_size=0.2)#cross-validation splitting

grid = GridSearchCV(GradientBoostingClassifier(), param_grid = param_grid, cv = cv, verbose = 0)#search for best set of hyperparameters
grid.fit(X, y)
grid.predict(X)
print(grid.best_score_)
print(grid.best_params_)


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
30 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py", line 525, in fit
    self._check_params()
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py", line 274, in _check_params
    raise ValueError(
ValueError: learning_rate must be greater than 0 but was 0.0

  warnings.warn(so

0.8699999999999999
{'learning_rate': 0.05, 'max_depth': 5}


In [ ]:
clf = GradientBoostingClassifier(max_depth=5, learning_rate = 0.05)
clf.fit(X, y)

GradientBoostingClassifier(learning_rate=0.05, max_depth=5)

In [ ]:
test = pd.read_csv("test.csv")
test.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [ ]:
df_test = test.drop(columns = ["RowNumber", "CustomerId", "Surname"])
df_test.isnull().any()

CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
dtype: bool

In [ ]:

df_test['Geography'] = le.transform(df_test['Geography'])
df_test = df_test.replace("Male", 0)
df_test = df_test.replace("Female", 1)

In [ ]:
y_test = clf.predict(df_test)
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
upload = pd.read_csv("upload.csv")
upload

,Unnamed: 0,RowNumber,Exited
0,0,2209,0
1,1,9924,0
2,2,4617,0
3,3,6077,0
4,4,9240,0
...,...,...,...
1995,1995,7872,0
1996,1996,4257,0
1997,1997,2273,0
1998,1998,315,0


In [ ]:
upload["Exited"] = y_test
upload

,Unnamed: 0,RowNumber,Exited
0,0,2209,0
1,1,9924,0
2,2,4617,0
3,3,6077,0
4,4,9240,1
...,...,...,...
1995,1995,7872,0
1996,1996,4257,0
1997,1997,2273,0
1998,1998,315,0


In [ ]:
upload.to_csv("upload_gbc.csv")